<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/06_Segmentation_Medical_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 06. Segmentation d'Images Médicales - Approches Comparatives

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Maîtriser les différentes approches de segmentation d'images médicales.

**Applications pratiques :**
- Segmentation manuelle par intensité et gradients
- Segmentation interactive avec MedSAM (Segment Anything Medical)
- Comparaison des approches manuelles vs automatiques
- Métriques quantitatives et évaluation de performance
- Applications cliniques spécialisées

**Important:** Ce cours compare les méthodes traditionnelles et modernes de segmentation médicale.

## Installation et Configuration

In [ ]:
# Installation des bibliothèques pour segmentation médicale
!pip install torch torchvision transformers datasets -q
!pip install opencv-python scikit-image matplotlib numpy pandas seaborn -q
!pip install pillow scipy monai -q
!pip install segment-anything-py -q

import torch
import torchvision
from transformers import SamModel, SamProcessor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from skimage import measure, morphology, filters, segmentation
from scipy import ndimage
from sklearn.metrics import jaccard_score, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Configuration système
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)

print(f"Dispositif utilisé: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("Configuration pour segmentation médicale terminée.")

## 1. Création d'Images Médicales Test

Créons des images médicales synthétiques avec structures anatomiques clairement définies pour tester nos méthodes de segmentation.

In [ ]:
# Génération d'images médicales synthétiques pour tests
print("=== GÉNÉRATION D'IMAGES MÉDICALES SYNTHÉTIQUES ===")

def create_medical_test_image(image_type='chest_xray', size=256, pathology=None):
    """
    Crée des images médicales synthétiques pour test de segmentation
    
    image_type: 'chest_xray', 'brain_mri', 'abdominal_ct'
    pathology: None, 'tumor', 'lesion', 'effusion'
    """
    img = np.zeros((size, size), dtype=np.float32)
    
    if image_type == 'chest_xray':
        # Création d'une radiographie thoracique
        
        # Poumons
        lung_intensity = 0.3
        
        # Poumon droit
        right_center_y, right_center_x = int(0.4 * size), int(0.25 * size)
        right_h, right_w = int(0.3 * size), int(0.15 * size)
        y_coords, x_coords = np.ogrid[:size, :size]
        right_mask = ((y_coords - right_center_y)**2 / right_h**2 + 
                     (x_coords - right_center_x)**2 / right_w**2) <= 1
        img[right_mask] = lung_intensity
        
        # Poumon gauche (plus petit anatomiquement)
        left_center_y, left_center_x = int(0.4 * size), int(0.75 * size)
        left_h, left_w = int(0.28 * size), int(0.15 * size)
        left_mask = ((y_coords - left_center_y)**2 / left_h**2 + 
                    (x_coords - left_center_x)**2 / left_w**2) <= 1
        img[left_mask] = lung_intensity
        
        # Cœur
        heart_center_y, heart_center_x = int(0.6 * size), int(0.48 * size)
        heart_h, heart_w = int(0.08 * size), int(0.12 * size)
        heart_mask = ((y_coords - heart_center_y)**2 / heart_h**2 + 
                     (x_coords - heart_center_x)**2 / heart_w**2) <= 1
        img[heart_mask] = 0.7
        
        # Rachis
        spine_width = int(0.03 * size)
        img[int(0.1*size):int(0.9*size), 
            int(0.5*size-spine_width):int(0.5*size+spine_width)] = 0.8
        
        # Côtes
        for rib_level in range(6):
            rib_y = int(0.2 * size) + rib_level * int(0.08 * size)
            for x in range(int(0.1 * size), int(0.9 * size)):
                if x < size // 2:
                    curve = int(0.03 * size * np.sin(np.pi * (x - 0.1 * size) / (0.4 * size)))
                else:
                    curve = int(0.03 * size * np.sin(np.pi * (0.9 * size - x) / (0.4 * size)))
                y_rib = rib_y + curve
                if 0 <= y_rib < size:
                    img[max(0, y_rib-1):min(size, y_rib+2), x:x+1] = 0.6
        
        # Pathologie spécifique
        if pathology == 'effusion':
            # Épanchement pleural droit
            effusion_height = int(0.2 * size)
            img[int(0.7*size):int(0.9*size), int(0.1*size):int(0.4*size)] = 0.8
        
        elif pathology == 'tumor':
            # Masse pulmonaire
            tumor_y, tumor_x = int(0.35 * size), int(0.3 * size)
            tumor_radius = int(0.04 * size)
            tumor_mask = ((y_coords - tumor_y)**2 + (x_coords - tumor_x)**2) <= tumor_radius**2
            img[tumor_mask] = 0.9
    
    elif image_type == 'brain_mri':
        # Création d'une IRM cérébrale
        
        # Crâne
        skull_center = size // 2
        skull_radius = int(0.4 * size)
        skull_mask = ((y_coords - skull_center)**2 + (x_coords - skull_center)**2) <= skull_radius**2
        img[skull_mask] = 0.2
        
        # Cerveau
        brain_radius = int(0.35 * size)
        brain_mask = ((y_coords - skull_center)**2 + (x_coords - skull_center)**2) <= brain_radius**2
        img[brain_mask] = 0.6
        
        # Ventricules
        ventricle_size = int(0.03 * size)
        ventricle1_mask = ((y_coords - skull_center + ventricle_size)**2 + 
                          (x_coords - skull_center)**2) <= ventricle_size**2
        ventricle2_mask = ((y_coords - skull_center - ventricle_size)**2 + 
                          (x_coords - skull_center)**2) <= ventricle_size**2
        img[ventricle1_mask | ventricle2_mask] = 0.9
        
        if pathology == 'tumor':
            # Tumeur cérébrale
            tumor_y, tumor_x = int(0.3 * size), int(0.4 * size)
            tumor_radius = int(0.05 * size)
            tumor_mask = ((y_coords - tumor_y)**2 + (x_coords - tumor_x)**2) <= tumor_radius**2
            img[tumor_mask] = 0.9
            
            # Œdème péritumoral
            edema_radius = int(0.08 * size)
            edema_mask = ((y_coords - tumor_y)**2 + (x_coords - tumor_x)**2) <= edema_radius**2
            edema_mask &= ~tumor_mask
            img[edema_mask] = 0.8
    
    # Post-traitement réaliste
    noise = np.random.normal(0, 0.02, img.shape)
    img += noise
    img = ndimage.gaussian_filter(img, sigma=0.8)
    img = np.clip(img, 0, 1)
    
    return img

# Création d'images test
test_images = {
    'chest_normal': {
        'image': create_medical_test_image('chest_xray'),
        'description': 'Radiographie thoracique normale',
        'structures': ['poumons', 'cœur', 'côtes', 'rachis']
    },
    'chest_effusion': {
        'image': create_medical_test_image('chest_xray', pathology='effusion'),
        'description': 'Radiographie avec épanchement pleural',
        'structures': ['poumons', 'cœur', 'épanchement']
    },
    'chest_tumor': {
        'image': create_medical_test_image('chest_xray', pathology='tumor'),
        'description': 'Radiographie avec masse pulmonaire',
        'structures': ['poumons', 'cœur', 'masse']
    },
    'brain_normal': {
        'image': create_medical_test_image('brain_mri'),
        'description': 'IRM cérébrale normale',
        'structures': ['crâne', 'cerveau', 'ventricules']
    },
    'brain_tumor': {
        'image': create_medical_test_image('brain_mri', pathology='tumor'),
        'description': 'IRM avec tumeur cérébrale',
        'structures': ['crâne', 'cerveau', 'tumeur', 'œdème']
    }
}

print(f"Images médicales test créées: {len(test_images)} types")

# Visualisation des images test
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Images Médicales Synthétiques pour Tests de Segmentation', 
            fontsize=16, fontweight='bold')

axes = axes.flatten()
for i, (img_name, img_data) in enumerate(test_images.items()):
    if i < len(axes):
        axes[i].imshow(img_data['image'], cmap='gray')
        axes[i].set_title(f"{img_data['description']}", fontweight='bold')
        axes[i].axis('off')
        
        # Ajouter liste des structures
        structures_text = ', '.join(img_data['structures'])
        axes[i].text(0.02, 0.02, f"Structures: {structures_text}", 
                    transform=axes[i].transAxes, fontsize=8,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
                    verticalalignment='bottom')

# Masquer l'axe vide
if len(test_images) < len(axes):
    axes[-1].axis('off')

plt.tight_layout()
plt.show()

print("Images test prêtes pour segmentation.")

## 2. Méthodes de Segmentation Manuelles

Implémentons les méthodes traditionnelles de segmentation basées sur l'intensité et les gradients.

In [ ]:
# Méthodes de segmentation manuelles/traditionnelles
print("=== MÉTHODES DE SEGMENTATION TRADITIONNELLES ===")

class TraditionalSegmentation:
    """
    Classe pour méthodes de segmentation traditionnelles
    """
    
    def __init__(self):
        self.method_names = [
            'Seuillage simple', 'Seuillage d\'Otsu', 'Watershed',
            'Contours actifs', 'Région croissante', 'K-means'
        ]
    
    def threshold_segmentation(self, image, threshold_method='otsu'):
        """
        Segmentation par seuillage
        """
        if threshold_method == 'otsu':
            threshold_value = filters.threshold_otsu(image)
        elif threshold_method == 'li':
            threshold_value = filters.threshold_li(image)
        elif threshold_method == 'yen':
            threshold_value = filters.threshold_yen(image)
        else:
            threshold_value = 0.5  # Seuil manuel
        
        binary_mask = image > threshold_value
        
        # Nettoyage morphologique
        cleaned_mask = morphology.remove_small_objects(binary_mask, min_size=100)
        cleaned_mask = morphology.remove_small_holes(cleaned_mask, area_threshold=50)
        
        return cleaned_mask, threshold_value
    
    def watershed_segmentation(self, image, markers=None):
        """
        Segmentation par ligne de partage des eaux (watershed)
        """
        # Préparation de l'image
        image_uint8 = (image * 255).astype(np.uint8)
        
        # Calcul du gradient
        gradient = filters.rank.gradient(image_uint8, morphology.disk(2))
        
        # Marqueurs automatiques si non fournis
        if markers is None:
            # Marqueurs basés sur les maxima locaux
            local_maxima = morphology.local_maxima(image)
            markers = measure.label(local_maxima)
        
        # Application de watershed
        labels = segmentation.watershed(gradient, markers)
        
        return labels, gradient
    
    def region_growing_segmentation(self, image, seed_points=None, tolerance=0.1):
        """
        Segmentation par croissance de régions
        """
        h, w = image.shape
        segmented = np.zeros((h, w), dtype=bool)
        
        if seed_points is None:
            # Points de départ automatiques (centres de masses locales)
            seed_points = [(h//4, w//4), (h//4, 3*w//4), (3*h//4, w//2)]
        
        for seed_y, seed_x in seed_points:
            if 0 <= seed_y < h and 0 <= seed_x < w:
                # Valeur de référence
                seed_value = image[seed_y, seed_x]
                
                # Croissance de région
                visited = np.zeros((h, w), dtype=bool)
                stack = [(seed_y, seed_x)]
                
                while stack:
                    y, x = stack.pop()
                    
                    if (0 <= y < h and 0 <= x < w and 
                        not visited[y, x] and 
                        abs(image[y, x] - seed_value) < tolerance):
                        
                        visited[y, x] = True
                        segmented[y, x] = True
                        
                        # Ajouter voisins
                        for dy, dx in [(-1,0), (1,0), (0,-1), (0,1)]:
                            stack.append((y+dy, x+dx))
        
        return segmented
    
    def kmeans_segmentation(self, image, n_clusters=3):
        """
        Segmentation par K-means clustering
        """
        from sklearn.cluster import KMeans
        
        # Reshape image pour K-means
        pixels = image.reshape(-1, 1)
        
        # Clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(pixels)
        
        # Reshape résultats
        segmented_image = labels.reshape(image.shape)
        
        return segmented_image, kmeans.cluster_centers_
    
    def edge_based_segmentation(self, image):
        """
        Segmentation basée sur la détection de contours
        """
        # Conversion pour OpenCV
        image_uint8 = (image * 255).astype(np.uint8)
        
        # Lissage
        blurred = cv2.GaussianBlur(image_uint8, (5, 5), 0)
        
        # Détection de contours
        edges = cv2.Canny(blurred, 50, 150)
        
        # Fermeture morphologique
        kernel = np.ones((3, 3), np.uint8)
        closed_edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=2)
        
        # Remplissage des contours
        contours, _ = cv2.findContours(closed_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        filled_mask = np.zeros_like(image_uint8)
        cv2.drawContours(filled_mask, contours, -1, 255, -1)
        
        return filled_mask > 0, edges

# Test des méthodes traditionnelles sur une image
segmenter = TraditionalSegmentation()
test_image = test_images['chest_normal']['image']

print("\nTest des méthodes traditionnelles sur radiographie thoracique...")

# Application des différentes méthodes
results_traditional = {}

# 1. Seuillage d'Otsu
otsu_mask, otsu_threshold = segmenter.threshold_segmentation(test_image, 'otsu')
results_traditional['Otsu'] = {
    'mask': otsu_mask,
    'info': f'Seuil: {otsu_threshold:.3f}'
}

# 2. Watershed
watershed_labels, gradient_img = segmenter.watershed_segmentation(test_image)
results_traditional['Watershed'] = {
    'mask': watershed_labels > 0,
    'info': f'Régions: {len(np.unique(watershed_labels))}'
}

# 3. Croissance de régions
region_growing_mask = segmenter.region_growing_segmentation(test_image)
results_traditional['Croissance'] = {
    'mask': region_growing_mask,
    'info': 'Tolérance: 0.1'
}

# 4. K-means
kmeans_labels, centroids = segmenter.kmeans_segmentation(test_image, n_clusters=3)
# Prendre la classe intermédiaire comme structure d'intérêt
kmeans_mask = kmeans_labels == 1
results_traditional['K-means'] = {
    'mask': kmeans_mask,
    'info': f'Classes: {len(centroids)}'
}

# 5. Contours
edge_mask, edge_img = segmenter.edge_based_segmentation(test_image)
results_traditional['Contours'] = {
    'mask': edge_mask,
    'info': 'Canny + fermeture'
}

print(f"Méthodes traditionnelles testées: {len(results_traditional)}")

# Visualisation comparative
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Comparaison des Méthodes de Segmentation Traditionnelles', 
            fontsize=16, fontweight='bold')

# Image originale
axes[0, 0].imshow(test_image, cmap='gray')
axes[0, 0].set_title('Image Originale', fontweight='bold')
axes[0, 0].axis('off')

# Résultats des méthodes
methods = list(results_traditional.keys())
for i, method in enumerate(methods):
    if i < 5:
        row = (i + 1) // 3
        col = (i + 1) % 3
        
        # Superposition du masque sur l'image originale
        axes[row, col].imshow(test_image, cmap='gray', alpha=0.7)
        axes[row, col].imshow(results_traditional[method]['mask'], 
                             cmap='Reds', alpha=0.5)
        axes[row, col].contour(results_traditional[method]['mask'], 
                              colors='red', linewidths=2)
        
        title = f"{method}\n{results_traditional[method]['info']}"
        axes[row, col].set_title(title, fontweight='bold')
        axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("Méthodes traditionnelles comparées avec succès.")

## 3. Segmentation Interactive avec MedSAM

Implémentons la segmentation interactive avec MedSAM (Segment Anything Model) spécialisé pour le médical.

In [ ]:
# Segmentation interactive avec MedSAM
print("=== SEGMENTATION INTERACTIVE AVEC MEDSAM ===")

class MedSAMSegmentation:
    """
    Classe pour segmentation interactive avec MedSAM
    """
    
    def __init__(self, model_name="wanglab/medsam-vit-base"):
        """
        Initialise le modèle MedSAM
        """
        try:
            print(f"Chargement du modèle MedSAM: {model_name}")
            self.processor = SamProcessor.from_pretrained(model_name)
            self.model = SamModel.from_pretrained(model_name).to(device)
            self.model.eval()
            print("MedSAM chargé avec succès")
        except Exception as e:
            print(f"Erreur chargement MedSAM: {e}")
            print("Utilisation d'un modèle SAM standard")
            try:
                from transformers import SamModel, SamProcessor
                self.processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
                self.model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)
                self.model.eval()
                print("SAM standard chargé comme alternative")
            except Exception as e2:
                print(f"Erreur SAM: {e2}")
                self.model = None
                self.processor = None
    
    def segment_with_box(self, image, bounding_box):
        """
        Segmentation avec boîte englobante
        
        bounding_box: [x_min, y_min, x_max, y_max]
        """
        if self.model is None:
            print("Modèle non disponible")
            return None, None
        
        # Préparation de l'image
        if len(image.shape) == 2:
            # Convertir grayscale en RGB
            image_rgb = np.stack([image] * 3, axis=-1)
        else:
            image_rgb = image
        
        # Conversion en PIL Image pour le processeur
        pil_image = Image.fromarray((image_rgb * 255).astype(np.uint8))
        
        try:
            # Préparation des entrées
            inputs = self.processor(
                pil_image, 
                input_boxes=[[bounding_box]], 
                return_tensors="pt"
            ).to(device)
            
            # Inférence
            with torch.no_grad():
                outputs = self.model(**inputs)
            
            # Post-traitement
            masks = outputs.pred_masks.squeeze(1)
            masks = torch.sigmoid(masks)
            masks = masks.cpu().numpy().squeeze()
            
            # Conversion en masque binaire
            binary_mask = (masks > 0.5).astype(np.uint8)
            
            # Calcul de la confiance
            confidence = float(masks.max())
            
            return binary_mask, confidence
            
        except Exception as e:
            print(f"Erreur lors de la segmentation: {e}")
            return None, None
    
    def segment_with_points(self, image, point_coords, point_labels):
        """
        Segmentation avec points de contrôle
        
        point_coords: liste de [x, y]
        point_labels: liste de 0 (négatif) ou 1 (positif)
        """
        if self.model is None:
            return None, None
        
        # Préparation de l'image
        if len(image.shape) == 2:
            image_rgb = np.stack([image] * 3, axis=-1)
        else:
            image_rgb = image
        
        pil_image = Image.fromarray((image_rgb * 255).astype(np.uint8))
        
        try:
            # Préparation des entrées
            inputs = self.processor(
                pil_image, 
                input_points=[point_coords],
                input_labels=[point_labels],
                return_tensors="pt"
            ).to(device)
            
            # Inférence
            with torch.no_grad():
                outputs = self.model(**inputs)
            
            # Post-traitement
            masks = outputs.pred_masks.squeeze(1)
            masks = torch.sigmoid(masks)
            masks = masks.cpu().numpy().squeeze()
            
            binary_mask = (masks > 0.5).astype(np.uint8)
            confidence = float(masks.max())
            
            return binary_mask, confidence
            
        except Exception as e:
            print(f"Erreur lors de la segmentation par points: {e}")
            return None, None
    
    def auto_generate_boxes(self, image, structure_type='lung'):
        """
        Génération automatique de boîtes englobantes pour structures communes
        """
        h, w = image.shape
        boxes = []
        
        if structure_type == 'lung':
            # Boîtes pour poumons gauche et droit
            # Poumon droit
            boxes.append([int(0.1*w), int(0.2*h), int(0.45*w), int(0.7*h)])
            # Poumon gauche
            boxes.append([int(0.55*w), int(0.2*h), int(0.9*w), int(0.7*h)])
        
        elif structure_type == 'heart':
            # Boîte pour cœur
            boxes.append([int(0.35*w), int(0.45*h), int(0.65*w), int(0.75*h)])
        
        elif structure_type == 'brain':
            # Boîte pour cerveau entier
            boxes.append([int(0.1*w), int(0.1*h), int(0.9*w), int(0.9*h)])
        
        return boxes

# Initialisation de MedSAM
medsam_segmenter = MedSAMSegmentation()

# Test de segmentation interactive
if medsam_segmenter.model is not None:
    print("\nTest de segmentation MedSAM...")
    
    # Test sur image thoracique
    test_image_chest = test_images['chest_normal']['image']
    
    # Génération de boîtes automatiques
    lung_boxes = medsam_segmenter.auto_generate_boxes(test_image_chest, 'lung')
    heart_box = medsam_segmenter.auto_generate_boxes(test_image_chest, 'heart')[0]
    
    results_medsam = {}
    
    # Segmentation des poumons
    for i, box in enumerate(lung_boxes):
        mask, conf = medsam_segmenter.segment_with_box(test_image_chest, box)
        if mask is not None:
            results_medsam[f'Poumon_{i+1}'] = {
                'mask': mask,
                'confidence': conf,
                'box': box
            }
    
    # Segmentation du cœur
    heart_mask, heart_conf = medsam_segmenter.segment_with_box(test_image_chest, heart_box)
    if heart_mask is not None:
        results_medsam['Cœur'] = {
            'mask': heart_mask,
            'confidence': heart_conf,
            'box': heart_box
        }
    
    # Test avec points de contrôle
    h, w = test_image_chest.shape
    lung_points = [[int(0.25*w), int(0.4*h)], [int(0.75*w), int(0.4*h)]]  # Points dans les poumons
    point_labels = [1, 1]  # Points positifs
    
    points_mask, points_conf = medsam_segmenter.segment_with_points(
        test_image_chest, lung_points, point_labels
    )
    
    if points_mask is not None:
        results_medsam['Points_contrôle'] = {
            'mask': points_mask,
            'confidence': points_conf,
            'points': lung_points
        }
    
    print(f"Segmentations MedSAM réussies: {len(results_medsam)}")

else:
    print("MedSAM non disponible, création de segmentations simulées")
    results_medsam = {}
    
    # Simulation de résultats MedSAM
    test_image_chest = test_images['chest_normal']['image']
    h, w = test_image_chest.shape
    
    # Masque simulé pour poumon droit
    lung1_mask = np.zeros((h, w), dtype=bool)
    lung1_mask[int(0.2*h):int(0.7*h), int(0.1*w):int(0.45*w)] = True
    lung1_mask = lung1_mask & (test_image_chest < 0.5)
    
    results_medsam['Poumon_1'] = {
        'mask': lung1_mask.astype(np.uint8),
        'confidence': 0.85,
        'box': [int(0.1*w), int(0.2*h), int(0.45*w), int(0.7*h)]
    }
    
    print("Segmentations MedSAM simulées créées")

## 4. Comparaison Quantitative des Méthodes

Comparons objectivement les performances des différentes approches de segmentation.

In [ ]:
# Comparaison quantitative des méthodes de segmentation
print("=== COMPARAISON QUANTITATIVE DES MÉTHODES ===")

def calculate_segmentation_metrics(mask1, mask2):
    """
    Calcule les métriques de comparaison entre deux masques
    """
    # Conversion en format binaire
    mask1_binary = mask1.astype(bool)
    mask2_binary = mask2.astype(bool)
    
    # Intersection et union
    intersection = np.logical_and(mask1_binary, mask2_binary)
    union = np.logical_or(mask1_binary, mask2_binary)
    
    # Métriques
    dice_coefficient = 2 * np.sum(intersection) / (np.sum(mask1_binary) + np.sum(mask2_binary))
    jaccard_index = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0
    
    # Sensibilité et spécificité
    tp = np.sum(intersection)
    fp = np.sum(mask1_binary) - tp
    fn = np.sum(mask2_binary) - tp
    tn = np.sum(~mask1_binary & ~mask2_binary)
    
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    
    return {
        'dice': dice_coefficient,
        'jaccard': jaccard_index,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision
    }

def create_ground_truth_mask(image, structure_type='lung'):
    """
    Crée un masque de vérité terrain simple pour évaluation
    """
    h, w = image.shape
    y_coords, x_coords = np.ogrid[:h, :w]
    
    if structure_type == 'lung':
        # Masque simplifié des poumons basé sur l'intensité et la position
        lung_mask = (image < 0.5) & (
            ((x_coords < w//2) & (y_coords > h//6) & (y_coords < 5*h//6)) |
            ((x_coords > w//2) & (y_coords > h//6) & (y_coords < 5*h//6))
        )
        # Nettoyage
        lung_mask = morphology.remove_small_objects(lung_mask, min_size=200)
        return lung_mask
    
    elif structure_type == 'heart':
        # Masque simplifié du cœur
        heart_center_y, heart_center_x = int(0.6 * h), int(0.5 * w)
        heart_h, heart_w = int(0.15 * h), int(0.2 * w)
        heart_mask = ((y_coords - heart_center_y)**2 / heart_h**2 + 
                     (x_coords - heart_center_x)**2 / heart_w**2) <= 1
        heart_mask = heart_mask & (image > 0.6)
        return heart_mask
    
    return np.zeros((h, w), dtype=bool)

# Création des masques de vérité terrain
test_image_chest = test_images['chest_normal']['image']
ground_truth_lung = create_ground_truth_mask(test_image_chest, 'lung')
ground_truth_heart = create_ground_truth_mask(test_image_chest, 'heart')

print("Masques de vérité terrain créés")

# Évaluation des méthodes traditionnelles
traditional_evaluation = {}

for method_name, result in results_traditional.items():
    metrics = calculate_segmentation_metrics(result['mask'], ground_truth_lung)
    traditional_evaluation[method_name] = metrics

print(f"\nÉvaluation des méthodes traditionnelles (vs poumons):")
print("-" * 60)
print(f"{'Méthode':<12} {'Dice':<6} {'Jaccard':<8} {'Sens.':<6} {'Spec.':<6} {'Prec.':<6}")
print("-" * 60)

for method, metrics in traditional_evaluation.items():
    print(f"{method:<12} {metrics['dice']:.3f}  {metrics['jaccard']:.3f}   "
          f"{metrics['sensitivity']:.3f}  {metrics['specificity']:.3f}  {metrics['precision']:.3f}")

# Évaluation MedSAM si disponible
if results_medsam:
    print(f"\nÉvaluation MedSAM:")
    print("-" * 60)
    
    for structure_name, result in results_medsam.items():
        if 'Poumon' in structure_name:
            metrics = calculate_segmentation_metrics(result['mask'], ground_truth_lung)
        elif 'Cœur' in structure_name:
            metrics = calculate_segmentation_metrics(result['mask'], ground_truth_heart)
        else:
            metrics = calculate_segmentation_metrics(result['mask'], ground_truth_lung)
        
        confidence = result.get('confidence', 0)
        print(f"{structure_name:<12} Dice: {metrics['dice']:.3f}, Conf: {confidence:.3f}")

# Analyse comparative des performances
performance_summary = []

for method, metrics in traditional_evaluation.items():
    performance_summary.append({
        'Method': f"Trad: {method}",
        'Type': 'Traditionnel',
        'Dice': metrics['dice'],
        'Jaccard': metrics['jaccard'],
        'Precision': metrics['precision']
    })

if results_medsam:
    for structure, result in results_medsam.items():
        if 'Poumon' in structure:
            metrics = calculate_segmentation_metrics(result['mask'], ground_truth_lung)
            performance_summary.append({
                'Method': f"MedSAM: {structure}",
                'Type': 'IA Interactive',
                'Dice': metrics['dice'],
                'Jaccard': metrics['jaccard'],
                'Precision': metrics['precision']
            })

# Création du DataFrame pour analyse
df_performance = pd.DataFrame(performance_summary)

print(f"\nRésumé comparatif des performances:")
print(df_performance.round(3))

# Identification de la meilleure méthode
if not df_performance.empty:
    best_dice = df_performance.loc[df_performance['Dice'].idxmax()]
    best_jaccard = df_performance.loc[df_performance['Jaccard'].idxmax()]
    
    print(f"\nMeilleures performances:")
    print(f"• Meilleur Dice: {best_dice['Method']} ({best_dice['Dice']:.3f})")
    print(f"• Meilleur Jaccard: {best_jaccard['Method']} ({best_jaccard['Jaccard']:.3f})")

print("\nComparaison quantitative terminée.")

## 5. Visualisation Comparative Avancée

Créons des visualisations complètes pour comparer toutes les approches de segmentation.

In [ ]:
# Visualisation comparative avancée
print("=== VISUALISATION COMPARATIVE AVANCÉE ===")

def create_comprehensive_comparison(image, traditional_results, medsam_results, ground_truth):
    """
    Crée une visualisation complète comparant toutes les méthodes
    """
    # Configuration de la figure
    n_traditional = len(traditional_results)
    n_medsam = len(medsam_results) if medsam_results else 0
    n_total = n_traditional + n_medsam + 2  # +2 pour original et ground truth
    
    # Calculer la grille optimale
    cols = min(4, n_total)
    rows = (n_total + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    elif cols == 1:
        axes = axes.reshape(-1, 1)
    
    # Aplatir les axes pour faciliter l'indexation
    axes_flat = axes.flatten()
    
    current_idx = 0
    
    # 1. Image originale
    axes_flat[current_idx].imshow(image, cmap='gray')
    axes_flat[current_idx].set_title('Image Originale', fontweight='bold', fontsize=12)
    axes_flat[current_idx].axis('off')
    current_idx += 1
    
    # 2. Vérité terrain
    axes_flat[current_idx].imshow(image, cmap='gray', alpha=0.7)
    axes_flat[current_idx].imshow(ground_truth, cmap='Greens', alpha=0.6)
    axes_flat[current_idx].contour(ground_truth, colors='green', linewidths=2)
    axes_flat[current_idx].set_title('Vérité Terrain', fontweight='bold', fontsize=12, color='green')
    axes_flat[current_idx].axis('off')
    current_idx += 1
    
    # 3. Méthodes traditionnelles
    colors_traditional = ['red', 'blue', 'orange', 'purple', 'brown']
    
    for i, (method_name, result) in enumerate(traditional_results.items()):
        if current_idx < len(axes_flat):
            color = colors_traditional[i % len(colors_traditional)]
            
            axes_flat[current_idx].imshow(image, cmap='gray', alpha=0.7)
            axes_flat[current_idx].imshow(result['mask'], 
                                         cmap='Reds' if color == 'red' else 'Blues', alpha=0.5)
            axes_flat[current_idx].contour(result['mask'], colors=color, linewidths=2)
            
            # Calculer métrique avec ground truth
            metrics = calculate_segmentation_metrics(result['mask'], ground_truth)
            
            title = f"{method_name}\nDice: {metrics['dice']:.3f}"
            axes_flat[current_idx].set_title(title, fontweight='bold', fontsize=10)
            axes_flat[current_idx].axis('off')
            current_idx += 1
    
    # 4. Méthodes MedSAM
    if medsam_results:
        for structure_name, result in medsam_results.items():
            if current_idx < len(axes_flat):
                axes_flat[current_idx].imshow(image, cmap='gray', alpha=0.7)
                axes_flat[current_idx].imshow(result['mask'], cmap='Purples', alpha=0.6)
                axes_flat[current_idx].contour(result['mask'], colors='purple', linewidths=2)
                
                # Afficher boîte englobante si disponible
                if 'box' in result:
                    box = result['box']
                    rect = plt.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1],
                                       linewidth=1, edgecolor='yellow', facecolor='none', 
                                       linestyle='--')
                    axes_flat[current_idx].add_patch(rect)
                
                # Afficher points si disponible
                if 'points' in result:
                    points = result['points']
                    for point in points:
                        axes_flat[current_idx].plot(point[0], point[1], 'r*', markersize=8)
                
                metrics = calculate_segmentation_metrics(result['mask'], ground_truth)
                confidence = result.get('confidence', 0)
                
                title = f"MedSAM: {structure_name}\nDice: {metrics['dice']:.3f}, Conf: {confidence:.3f}"
                axes_flat[current_idx].set_title(title, fontweight='bold', fontsize=10, color='purple')
                axes_flat[current_idx].axis('off')
                current_idx += 1
    
    # Masquer les axes inutilisés
    for idx in range(current_idx, len(axes_flat)):
        axes_flat[idx].axis('off')
    
    plt.suptitle('Comparaison Complète des Méthodes de Segmentation', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Graphique de performance comparative
def create_performance_charts(df_performance):
    """
    Crée des graphiques de performance comparative
    """
    if df_performance.empty:
        print("Aucune donnée de performance disponible")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Graphique en barres des scores Dice
    df_sorted = df_performance.sort_values('Dice', ascending=True)
    
    colors = ['red' if 'Trad' in method else 'purple' for method in df_sorted['Method']]
    bars = axes[0, 0].barh(df_sorted['Method'], df_sorted['Dice'], color=colors, alpha=0.7)
    axes[0, 0].set_xlabel('Score Dice')
    axes[0, 0].set_title('Comparaison des Scores Dice', fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Ajouter les valeurs sur les barres
    for bar, value in zip(bars, df_sorted['Dice']):
        width = bar.get_width()
        axes[0, 0].text(width + 0.01, bar.get_y() + bar.get_height()/2,
                       f'{value:.3f}', ha='left', va='center', fontweight='bold')
    
    # 2. Graphique en barres des scores Jaccard
    df_sorted_jaccard = df_performance.sort_values('Jaccard', ascending=True)
    colors_j = ['red' if 'Trad' in method else 'purple' for method in df_sorted_jaccard['Method']]
    
    bars_j = axes[0, 1].barh(df_sorted_jaccard['Method'], df_sorted_jaccard['Jaccard'], 
                           color=colors_j, alpha=0.7)
    axes[0, 1].set_xlabel('Score Jaccard')
    axes[0, 1].set_title('Comparaison des Scores Jaccard', fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Scatter plot Dice vs Jaccard
    traditional_mask = df_performance['Type'] == 'Traditionnel'
    ai_mask = df_performance['Type'] == 'IA Interactive'
    
    if traditional_mask.any():
        axes[1, 0].scatter(df_performance[traditional_mask]['Dice'], 
                          df_performance[traditional_mask]['Jaccard'], 
                          c='red', alpha=0.7, s=100, label='Traditionnel', edgecolors='black')
    
    if ai_mask.any():
        axes[1, 0].scatter(df_performance[ai_mask]['Dice'], 
                          df_performance[ai_mask]['Jaccard'], 
                          c='purple', alpha=0.7, s=100, label='IA Interactive', edgecolors='black')
    
    axes[1, 0].set_xlabel('Score Dice')
    axes[1, 0].set_ylabel('Score Jaccard')
    axes[1, 0].set_title('Corrélation Dice vs Jaccard', fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Ligne de corrélation parfaite
    max_val = max(df_performance['Dice'].max(), df_performance['Jaccard'].max())
    axes[1, 0].plot([0, max_val], [0, max_val], 'k--', alpha=0.5, label='Corrélation parfaite')
    
    # 4. Comparaison par type de méthode
    if len(df_performance['Type'].unique()) > 1:
        df_grouped = df_performance.groupby('Type')[['Dice', 'Jaccard', 'Precision']].mean()
        
        x_pos = np.arange(len(df_grouped.index))
        width = 0.25
        
        axes[1, 1].bar(x_pos - width, df_grouped['Dice'], width, label='Dice', alpha=0.8)
        axes[1, 1].bar(x_pos, df_grouped['Jaccard'], width, label='Jaccard', alpha=0.8)
        axes[1, 1].bar(x_pos + width, df_grouped['Precision'], width, label='Precision', alpha=0.8)
        
        axes[1, 1].set_xlabel('Type de Méthode')
        axes[1, 1].set_ylabel('Score Moyen')
        axes[1, 1].set_title('Performance Moyenne par Type', fontweight='bold')
        axes[1, 1].set_xticks(x_pos)
        axes[1, 1].set_xticklabels(df_grouped.index)
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'Comparaison par type\nnon disponible\n(un seul type)', 
                        ha='center', va='center', transform=axes[1, 1].transAxes,
                        fontsize=12, bbox=dict(boxstyle="round", facecolor='lightgray'))
        axes[1, 1].set_xlim(0, 1)
        axes[1, 1].set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()

# Génération des visualisations
print("\nGénération de la comparaison visuelle complète...")

create_comprehensive_comparison(
    test_image_chest, 
    results_traditional, 
    results_medsam, 
    ground_truth_lung
)

print("\nGénération des graphiques de performance...")
create_performance_charts(df_performance)

print("\nVisualisation comparative terminée.")

## 6. Applications Cliniques Spécialisées

Développons des applications cliniques concrètes utilisant les meilleures méthodes de segmentation identifiées.

In [ ]:
# Applications cliniques spécialisées
print("=== APPLICATIONS CLINIQUES SPÉCIALISÉES ===")

class ClinicalSegmentationApplication:
    """
    Applications cliniques de segmentation médicale
    """
    
    def __init__(self, best_traditional_method, medsam_segmenter=None):
        self.best_traditional = best_traditional_method
        self.medsam = medsam_segmenter
        
    def calculate_cardiothoracic_ratio(self, chest_image):
        """
        Calcul automatique du ratio cardio-thoracique
        """
        h, w = chest_image.shape
        
        # Segmentation du cœur et des poumons
        if self.medsam and self.medsam.model is not None:
            # Utilisation de MedSAM
            heart_box = [int(0.35*w), int(0.45*h), int(0.65*w), int(0.75*h)]
            heart_mask, heart_conf = self.medsam.segment_with_box(chest_image, heart_box)
            
            lung_boxes = self.medsam.auto_generate_boxes(chest_image, 'lung')
            lung_mask_combined = np.zeros_like(chest_image, dtype=bool)
            
            for box in lung_boxes:
                lung_mask, _ = self.medsam.segment_with_box(chest_image, box)
                if lung_mask is not None:
                    lung_mask_combined |= lung_mask.astype(bool)
        else:
            # Utilisation de méthode traditionnelle
            segmenter = TraditionalSegmentation()
            lung_mask_combined, _ = segmenter.threshold_segmentation(chest_image, 'otsu')
            
            # Segmentation cardiaque simple par région d'intérêt
            heart_roi = chest_image[int(0.45*h):int(0.75*h), int(0.35*w):int(0.65*w)]
            heart_threshold = np.percentile(heart_roi, 70)
            heart_mask = np.zeros_like(chest_image, dtype=bool)
            heart_mask[int(0.45*h):int(0.75*h), int(0.35*w):int(0.65*w)] = heart_roi > heart_threshold
        
        # Calcul des diamètres
        if isinstance(heart_mask, np.ndarray) and heart_mask.any():
            # Diamètre cardiaque (largeur maximale)
            heart_coords = np.where(heart_mask)
            if len(heart_coords[1]) > 0:
                cardiac_width = np.max(heart_coords[1]) - np.min(heart_coords[1])
            else:
                cardiac_width = 0
        else:
            cardiac_width = 0
        
        # Diamètre thoracique (largeur des poumons)
        lung_coords = np.where(lung_mask_combined)
        if len(lung_coords[1]) > 0:
            thoracic_width = np.max(lung_coords[1]) - np.min(lung_coords[1])
        else:
            thoracic_width = w  # Fallback
        
        # Calcul du CTR
        ctr = cardiac_width / thoracic_width if thoracic_width > 0 else 0
        
        return {
            'ctr': ctr,
            'cardiac_width': cardiac_width,
            'thoracic_width': thoracic_width,
            'heart_mask': heart_mask if isinstance(heart_mask, np.ndarray) else np.zeros_like(chest_image, dtype=bool),
            'lung_mask': lung_mask_combined,
            'interpretation': self._interpret_ctr(ctr)
        }
    
    def _interpret_ctr(self, ctr):
        """
        Interprétation clinique du ratio cardio-thoracique
        """
        if ctr < 0.5:
            return "Normal (CTR < 0.5)"
        elif ctr < 0.6:
            return "Limite (0.5 ≤ CTR < 0.6) - Surveillance recommandée"
        else:
            return "Anormal (CTR ≥ 0.6) - Cardiomégalie probable"
    
    def tumor_volume_estimation(self, image, tumor_region):
        """
        Estimation du volume tumoral à partir de segmentation
        """
        if self.medsam and self.medsam.model is not None:
            # Utilisation de MedSAM pour segmentation précise
            tumor_mask, confidence = self.medsam.segment_with_box(image, tumor_region)
        else:
            # Fallback avec méthode traditionnelle
            x_min, y_min, x_max, y_max = tumor_region
            roi = image[y_min:y_max, x_min:x_max]
            
            # Seuillage adaptatif pour tumeur (généralement plus dense)
            threshold = np.percentile(roi, 75)
            tumor_mask_roi = roi > threshold
            
            tumor_mask = np.zeros_like(image, dtype=bool)
            tumor_mask[y_min:y_max, x_min:x_max] = tumor_mask_roi
            confidence = 0.7  # Confiance simulée
        
        if tumor_mask is not None and np.any(tumor_mask):
            # Calcul des métriques tumorales
            tumor_area = np.sum(tumor_mask)  # En pixels
            
            # Estimation du diamètre équivalent
            equivalent_diameter = 2 * np.sqrt(tumor_area / np.pi)
            
            # Propriétés morphologiques
            props = measure.regionprops(tumor_mask.astype(int))
            if props:
                prop = props[0]
                eccentricity = prop.eccentricity
                solidity = prop.solidity
                extent = prop.extent
            else:
                eccentricity = solidity = extent = 0
            
            return {
                'area_pixels': tumor_area,
                'equivalent_diameter': equivalent_diameter,
                'eccentricity': eccentricity,
                'solidity': solidity,
                'extent': extent,
                'confidence': confidence,
                'tumor_mask': tumor_mask,
                'risk_assessment': self._assess_tumor_risk(tumor_area, eccentricity, solidity)
            }
        else:
            return None
    
    def _assess_tumor_risk(self, area, eccentricity, solidity):
        """
        Évaluation du risque basée sur les caractéristiques morphologiques
        """
        risk_factors = []
        
        if area > 1000:  # Grande taille
            risk_factors.append("Taille importante")
        
        if eccentricity > 0.8:  # Forme allongée
            risk_factors.append("Forme irrégulière")
        
        if solidity < 0.8:  # Contours irréguliers
            risk_factors.append("Contours irréguliers")
        
        if len(risk_factors) == 0:
            return "Faible risque", risk_factors
        elif len(risk_factors) <= 1:
            return "Risque modéré", risk_factors
        else:
            return "Risque élevé", risk_factors
    
    def lung_symmetry_analysis(self, chest_image):
        """
        Analyse de la symétrie pulmonaire
        """
        h, w = chest_image.shape
        
        # Segmentation des poumons
        if self.medsam and self.medsam.model is not None:
            lung_boxes = self.medsam.auto_generate_boxes(chest_image, 'lung')
            left_mask = right_mask = None
            
            for i, box in enumerate(lung_boxes):
                mask, _ = self.medsam.segment_with_box(chest_image, box)
                if mask is not None:
                    if i == 0:  # Premier poumon (droit)
                        right_mask = mask
                    else:  # Deuxième poumon (gauche)
                        left_mask = mask
        else:
            # Méthode traditionnelle
            segmenter = TraditionalSegmentation()
            lung_mask_total, _ = segmenter.threshold_segmentation(chest_image, 'otsu')
            
            # Séparation gauche/droite
            mid_line = w // 2
            left_mask = lung_mask_total.copy()
            left_mask[:, :mid_line] = False
            
            right_mask = lung_mask_total.copy()
            right_mask[:, mid_line:] = False
        
        # Calcul des métriques de symétrie
        if left_mask is not None and right_mask is not None:
            left_area = np.sum(left_mask)
            right_area = np.sum(right_mask)
            
            # Index de symétrie
            if left_area + right_area > 0:
                symmetry_index = 2 * min(left_area, right_area) / (left_area + right_area)
            else:
                symmetry_index = 0
            
            # Asymétrie relative
            asymmetry = abs(left_area - right_area) / max(left_area, right_area) if max(left_area, right_area) > 0 else 0
            
            return {
                'left_area': left_area,
                'right_area': right_area,
                'symmetry_index': symmetry_index,
                'asymmetry': asymmetry,
                'left_mask': left_mask,
                'right_mask': right_mask,
                'interpretation': self._interpret_symmetry(symmetry_index)
            }
        
        return None
    
    def _interpret_symmetry(self, symmetry_index):
        """
        Interprétation clinique de la symétrie pulmonaire
        """
        if symmetry_index > 0.9:
            return "Symétrie excellente"
        elif symmetry_index > 0.8:
            return "Symétrie bonne"
        elif symmetry_index > 0.6:
            return "Asymétrie légère - Surveillance"
        else:
            return "Asymétrie significative - Investigation requise"

# Initialisation de l'application clinique
# Sélection de la meilleure méthode traditionnelle
if not df_performance.empty:
    best_traditional_row = df_performance[df_performance['Type'] == 'Traditionnel']
    if not best_traditional_row.empty:
        best_method_name = best_traditional_row.loc[best_traditional_row['Dice'].idxmax(), 'Method']
        best_method_name = best_method_name.replace('Trad: ', '')
    else:
        best_method_name = 'Otsu'
else:
    best_method_name = 'Otsu'

clinical_app = ClinicalSegmentationApplication(best_method_name, medsam_segmenter)

print(f"Application clinique initialisée avec méthode: {best_method_name}")

# Test des applications cliniques
print("\nTest des applications cliniques...")

# 1. Calcul du ratio cardio-thoracique
ctr_results = clinical_app.calculate_cardiothoracic_ratio(test_images['chest_normal']['image'])
print(f"\nRatio Cardio-Thoracique:")
print(f"  CTR: {ctr_results['ctr']:.3f}")
print(f"  Interprétation: {ctr_results['interpretation']}")
print(f"  Largeur cardiaque: {ctr_results['cardiac_width']} pixels")
print(f"  Largeur thoracique: {ctr_results['thoracic_width']} pixels")

# 2. Analyse de symétrie pulmonaire
symmetry_results = clinical_app.lung_symmetry_analysis(test_images['chest_normal']['image'])
if symmetry_results:
    print(f"\nSymétrie Pulmonaire:")
    print(f"  Index de symétrie: {symmetry_results['symmetry_index']:.3f}")
    print(f"  Interprétation: {symmetry_results['interpretation']}")
    print(f"  Aire poumon gauche: {symmetry_results['left_area']} pixels")
    print(f"  Aire poumon droit: {symmetry_results['right_area']} pixels")

# 3. Analyse tumorale (si image avec tumeur disponible)
if 'chest_tumor' in test_images:
    tumor_image = test_images['chest_tumor']['image']
    h, w = tumor_image.shape
    # Région d'intérêt approximative pour la tumeur
    tumor_region = [int(0.2*w), int(0.3*h), int(0.4*w), int(0.5*h)]
    
    tumor_results = clinical_app.tumor_volume_estimation(tumor_image, tumor_region)
    if tumor_results:
        print(f"\nAnalyse Tumorale:")
        print(f"  Aire: {tumor_results['area_pixels']} pixels")
        print(f"  Diamètre équivalent: {tumor_results['equivalent_diameter']:.1f} pixels")
        print(f"  Excentricité: {tumor_results['eccentricity']:.3f}")
        print(f"  Solidité: {tumor_results['solidity']:.3f}")
        risk_level, risk_factors = tumor_results['risk_assessment']
        print(f"  Évaluation du risque: {risk_level}")
        if risk_factors:
            print(f"  Facteurs de risque: {', '.join(risk_factors)}")

print("\nApplications cliniques testées avec succès.")

## 7. Rapport Clinique Automatisé

Générons un rapport clinique complet intégrant tous les résultats de segmentation.

In [ ]:
# Génération de rapport clinique automatisé
print("=== GÉNÉRATION DE RAPPORT CLINIQUE AUTOMATISÉ ===")

def generate_comprehensive_clinical_report(image_type, ctr_results, symmetry_results, 
                                         tumor_results=None, method_performance=None):
    """
    Génère un rapport clinique complet basé sur l'analyse de segmentation
    """
    from datetime import datetime
    
    report = f"""
{'='*80}
                    RAPPORT D'ANALYSE DE SEGMENTATION MÉDICALE
{'='*80}

Date d'analyse: {datetime.now().strftime('%d/%m/%Y à %H:%M')}
Type d'examen: {image_type.replace('_', ' ').title()}
Système d'analyse: IA de Segmentation Médicale Comparative

RÉSUMÉ EXÉCUTIF:
{'='*50}
"""
    
    # Analyse du ratio cardio-thoracique
    if ctr_results:
        ctr_status = "🟢 NORMAL" if ctr_results['ctr'] < 0.5 else "🔴 ANORMAL" if ctr_results['ctr'] >= 0.6 else "🟡 LIMITE"
        report += f"""
RATIO CARDIO-THORACIQUE:
• Valeur mesurée: {ctr_results['ctr']:.3f}
• Status: {ctr_status}
• Interprétation: {ctr_results['interpretation']}
"""
    
    # Analyse de symétrie
    if symmetry_results:
        sym_status = "🟢 NORMALE" if symmetry_results['symmetry_index'] > 0.8 else "🔴 ANORMALE"
        report += f"""
SYMÉTRIE PULMONAIRE:
• Index de symétrie: {symmetry_results['symmetry_index']:.3f}
• Status: {sym_status}
• Interprétation: {symmetry_results['interpretation']}
"""
    
    # Analyse tumorale si disponible
    if tumor_results:
        risk_level, risk_factors = tumor_results['risk_assessment']
        risk_status = "🟢 FAIBLE" if "Faible" in risk_level else "🔴 ÉLEVÉ" if "élevé" in risk_level else "🟡 MODÉRÉ"
        report += f"""
ANALYSE TUMORALE:
• Aire détectée: {tumor_results['area_pixels']} pixels
• Diamètre équivalent: {tumor_results['equivalent_diameter']:.1f} pixels
• Niveau de risque: {risk_status}
• Facteurs de risque: {', '.join(risk_factors) if risk_factors else 'Aucun'}
"""
    
    report += f"""

ANALYSE DÉTAILLÉE:
{'='*50}
"""
    
    # Détails de l'analyse
    if ctr_results:
        report += f"""
1. ANALYSE CARDIAQUE:
   - Largeur cardiaque mesurée: {ctr_results['cardiac_width']} pixels
   - Largeur thoracique mesurée: {ctr_results['thoracic_width']} pixels
   - Ratio calculé: {ctr_results['cardiac_width']}/{ctr_results['thoracic_width']} = {ctr_results['ctr']:.3f}
   
   INTERPRÉTATION CLINIQUE:
   - Normal: CTR < 0.5
   - Limite: 0.5 ≤ CTR < 0.6 (surveillance recommandée)
   - Anormal: CTR ≥ 0.6 (cardiomégalie probable)
   
   RÉSULTAT: {ctr_results['interpretation']}
"""
    
    if symmetry_results:
        asymmetry_percent = symmetry_results['asymmetry'] * 100
        larger_lung = "gauche" if symmetry_results['left_area'] > symmetry_results['right_area'] else "droit"
        
        report += f"""

2. ANALYSE PULMONAIRE:
   - Aire poumon gauche: {symmetry_results['left_area']} pixels
   - Aire poumon droit: {symmetry_results['right_area']} pixels
   - Asymétrie relative: {asymmetry_percent:.1f}%
   - Poumon prédominant: {larger_lung}
   
   INTERPRÉTATION CLINIQUE:
   - Excellente: Index > 0.9
   - Bonne: 0.8 < Index ≤ 0.9
   - Asymétrie légère: 0.6 < Index ≤ 0.8 (surveillance)
   - Asymétrie significative: Index ≤ 0.6 (investigation requise)
   
   RÉSULTAT: {symmetry_results['interpretation']}
"""
    
    if tumor_results:
        report += f"""

3. ANALYSE LÉSIONNELLE:
   - Aire de la lésion: {tumor_results['area_pixels']} pixels
   - Diamètre équivalent: {tumor_results['equivalent_diameter']:.1f} pixels
   - Excentricité: {tumor_results['eccentricity']:.3f} (0=cercle, 1=ligne)
   - Solidité: {tumor_results['solidity']:.3f} (régularité contours)
   - Extent: {tumor_results['extent']:.3f} (compacité)
   - Confiance segmentation: {tumor_results['confidence']:.3f}
   
   ÉVALUATION DU RISQUE:
   {risk_level}: {', '.join(risk_factors) if risk_factors else 'Caractéristiques morphologiques favorables'}
"""
    
    # Performance des méthodes
    if method_performance is not None and not method_performance.empty:
        best_method = method_performance.loc[method_performance['Dice'].idxmax()]
        report += f"""

PERFORMANCE DES MÉTHODES:
{'='*50}

Méthode la plus performante: {best_method['Method']}
• Score Dice: {best_method['Dice']:.3f}
• Score Jaccard: {best_method['Jaccard']:.3f}
• Précision: {best_method['Precision']:.3f}

Comparaison des approches:
"""
        
        traditional_avg = method_performance[method_performance['Type'] == 'Traditionnel']['Dice'].mean()
        ai_methods = method_performance[method_performance['Type'] == 'IA Interactive']
        
        report += f"• Méthodes traditionnelles (moyenne): {traditional_avg:.3f}\n"
        
        if not ai_methods.empty:
            ai_avg = ai_methods['Dice'].mean()
            report += f"• Méthodes IA interactives (moyenne): {ai_avg:.3f}\n"
    
    # Recommandations cliniques
    report += f"""

RECOMMANDATIONS CLINIQUES:
{'='*50}
"""
    
    recommendations = []
    
    if ctr_results and ctr_results['ctr'] >= 0.6:
        recommendations.append("🔴 URGENT: Échocardiographie recommandée (cardiomégalie)")
        recommendations.append("📞 Consultation cardiologique en priorité")
    elif ctr_results and ctr_results['ctr'] >= 0.5:
        recommendations.append("🟡 Surveillance: Contrôle radiologique à 6 mois")
    
    if symmetry_results and symmetry_results['symmetry_index'] <= 0.6:
        recommendations.append("🔴 Investigation: Scanner thoracique recommandé (asymétrie)")
        recommendations.append("🔬 Recherche de pathologie unilatérale")
    
    if tumor_results:
        risk_level, _ = tumor_results['risk_assessment']
        if "élevé" in risk_level:
            recommendations.append("🔴 URGENT: Biopsie et staging oncologique")
            recommendations.append("🏥 Réunion de concertation pluridisciplinaire (RCP)")
        elif "modéré" in risk_level:
            recommendations.append("🟡 Surveillance: Scanner de contrôle à 3 mois")
    
    if not recommendations:
        recommendations.append("🟢 Aucune anomalie significative détectée")
        recommendations.append("📋 Suivi standard selon protocole clinique")
    
    for rec in recommendations:
        report += f"\n{rec}"
    
    # Limitations et avertissements
    report += f"""


LIMITATIONS ET AVERTISSEMENTS:
{'='*50}

⚠️ IMPORTANT:
• Cette analyse est automatisée et doit être validée par un radiologue
• Les mesures sont en pixels, pas en unités anatomiques absolues
• La qualité de segmentation dépend de la qualité de l'image
• Toujours corréler avec l'examen clinique et l'anamnèse
• L'IA est un outil d'aide au diagnostic, non substitutif à l'expertise médicale

VALIDATION REQUISE:
• Contrôle visuel de la segmentation par le radiologue
• Vérification des mesures avec les outils PACS
• Corrélation avec les données cliniques du patient

{'='*80}
Rapport généré automatiquement par Système de Segmentation Médicale IA
Version: Démonstration Éducative
Statut: NON VALIDÉ - NÉCESSITE VALIDATION MÉDICALE
{'='*80}
"""
    
    return report

# Génération du rapport clinique complet
print("\nGénération du rapport clinique complet...")

clinical_report = generate_comprehensive_clinical_report(
    'radiographie_thoracique',
    ctr_results,
    symmetry_results,
    tumor_results=None,  # Pas de tumeur dans l'image normale
    method_performance=df_performance
)

print(clinical_report)

# Sauvegarde du rapport
with open('rapport_segmentation_medical.txt', 'w', encoding='utf-8') as f:
    f.write(clinical_report)

print("\n" + "="*60)
print("RAPPORT CLINIQUE SAUVEGARDÉ: rapport_segmentation_medical.txt")
print("="*60)

# Visualisation finale récapitulative
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Résumé Clinique - Analyse de Segmentation Médicale', 
            fontsize=16, fontweight='bold')

# Image originale
axes[0, 0].imshow(test_images['chest_normal']['image'], cmap='gray')
axes[0, 0].set_title('Image Originale', fontweight='bold')
axes[0, 0].axis('off')

# CTR analysis
axes[0, 1].imshow(test_images['chest_normal']['image'], cmap='gray', alpha=0.7)
if 'heart_mask' in ctr_results:
    axes[0, 1].contour(ctr_results['heart_mask'], colors='red', linewidths=2)
axes[0, 1].contour(ctr_results['lung_mask'], colors='blue', linewidths=2, linestyles='--')
axes[0, 1].set_title(f"Analyse CTR\nRatio: {ctr_results['ctr']:.3f}", fontweight='bold')
axes[0, 1].axis('off')

# Symmetry analysis
if symmetry_results:
    axes[1, 0].imshow(test_images['chest_normal']['image'], cmap='gray', alpha=0.7)
    axes[1, 0].contour(symmetry_results['left_mask'], colors='green', linewidths=2)
    axes[1, 0].contour(symmetry_results['right_mask'], colors='red', linewidths=2)
    axes[1, 0].axvline(x=test_images['chest_normal']['image'].shape[1]//2, 
                      color='yellow', linestyle=':', linewidth=2)
    axes[1, 0].set_title(f"Analyse Symétrie\nIndex: {symmetry_results['symmetry_index']:.3f}", 
                        fontweight='bold')
    axes[1, 0].axis('off')

# Performance summary
if not df_performance.empty:
    methods = df_performance['Method'].str.replace('Trad: ', '').str.replace('MedSAM: ', '')
    dice_scores = df_performance['Dice']
    
    bars = axes[1, 1].bar(range(len(methods)), dice_scores, 
                         color=['red' if 'Trad' in m else 'purple' for m in df_performance['Method']], 
                         alpha=0.7)
    axes[1, 1].set_xticks(range(len(methods)))
    axes[1, 1].set_xticklabels(methods, rotation=45, ha='right')
    axes[1, 1].set_ylabel('Score Dice')
    axes[1, 1].set_title('Performance des Méthodes', fontweight='bold')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Ajouter valeurs sur barres
    for bar, score in zip(bars, dice_scores):
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                        f'{score:.3f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

print("\nAnalyse complète de segmentation médicale terminée.")

## Résumé et Applications Futures

### Compétences Acquises

Dans ce notebook complet, vous avez maîtrisé:

1. **Méthodes de Segmentation Traditionnelles**
   - Seuillage adaptatif (Otsu, Li, Yen)
   - Segmentation par watershed
   - Croissance de régions
   - K-means clustering
   - Segmentation basée sur les contours

2. **Segmentation Interactive avec MedSAM**
   - Utilisation de boîtes englobantes pour guidage
   - Segmentation par points de contrôle
   - Génération automatique de régions d'intérêt
   - Post-traitement et optimisation des résultats

3. **Évaluation Quantitative Comparative**
   - Métriques de performance (Dice, Jaccard, sensibilité, spécificité)
   - Comparaison objective des approches
   - Analyse statistique des résultats
   - Identification des meilleures méthodes par contexte

4. **Applications Cliniques Spécialisées**
   - Calcul automatique du ratio cardio-thoracique
   - Analyse de symétrie pulmonaire
   - Estimation volumétrique tumorale
   - Évaluation morphologique des lésions

5. **Génération de Rapports Automatisés**
   - Rapports cliniques structurés
   - Interprétation automatique des résultats
   - Recommandations thérapeutiques adaptées
   - Intégration dans workflows hospitaliers

### Applications Médicales Directes

Ces compétences vous permettront de:
- **Choisir la méthode optimale** selon le type d'image et la pathologie
- **Implémenter des systèmes hybrides** combinant approches traditionnelles et IA
- **Quantifier précisément** les structures anatomiques
- **Automatiser les mesures cliniques** standardisées
- **Générer des rapports** d'aide au diagnostic

### Recommandations d'Usage Clinique

1. **Sélection de Méthodes**:
   - **Méthodes traditionnelles**: Images de qualité variable, contraintes computationnelles
   - **MedSAM**: Segmentation interactive précise, structures complexes
   - **Approche hybride**: Combinaison pour robustesse maximale

2. **Validation Clinique**:
   - Toujours vérifier visuellement les segmentations
   - Valider les métriques avec mesures manuelles
   - Adapter les seuils selon les populations patients

3. **Intégration Workflow**:
   - Automatisation des mesures de routine
   - Alertes pour valeurs anormales
   - Traçabilité des analyses automatisées

### Limitations et Perspectives

- **Qualité d'image**: Performance dépendante de la qualité d'acquisition
- **Variabilité anatomique**: Adaptation nécessaire selon populations
- **Validation continue**: Surveillance des performances en conditions réelles
- **Évolution technologique**: Intégration de nouvelles méthodes IA

### Prochaine Étape

Le prochain notebook vous enseignera l'**implémentation complète de nnU-Net** pour segmentation automatique avancée, incluant l'entraînement sur données personnalisées et l'intégration dans des workflows cliniques.